In [35]:
import pandas as pd
pd.set_option('display.max_columns', None)

# Load the SraRunTable.txt file with the correct delimiter
sra_run_table = pd.read_csv('SraRunTable.txt', sep=',')
# Extract relevant columns
metadata = sra_run_table[['Run', 'GEO_Accession (exp)', 'treatment']]

metadata
# Rename columns to match the required format
#metadata.columns = ['SRR_ID', 'GSM_ID', 'Timepoint']

# Save to CSV
#metadata.to_csv('metadata.csv', index=False)


,Run,GEO_Accession (exp),treatment
0,SRR5739552,GSM2678508,WT
1,SRR5739553,GSM2678509,WT
2,SRR5739554,GSM2678510,WT
3,SRR5739555,GSM2678511,WT
4,SRR5739556,GSM2678512,WT
...,...,...,...
475,SRR5740027,GSM2678983,BJ fibroblasts 16 days post induced viral expr...
476,SRR5740028,GSM2678984,BJ fibroblasts 16 days post induced viral expr...
477,SRR5740029,GSM2678985,BJ fibroblasts 16 days post induced viral expr...
478,SRR5740030,GSM2678986,BJ fibroblasts 16 days post induced viral expr...


In [30]:
import pandas as pd

# Load the SraRunTable.txt file with the correct delimiter
sra_run_table = pd.read_csv('SraRunTable.txt', sep=',')

# Extract relevant columns
metadata = sra_run_table[['Run', 'treatment', 'tra-1-60']]

# Generate the sample_name column
def generate_sample_name(row, id):
    treatment_words = row['treatment'].split()[:3]
    sample_name = '_'.join(treatment_words) + f'_{id}'
    if pd.notna(row['tra-1-60']) and row['tra-1-60'].strip() != '':
        sample_name += f"_{row['tra-1-60']}"
    return sample_name

# Add the sample_name column
sample_names = []
for idx, row in metadata.iterrows():
    id = (idx % 96) + 1
    sample_name = generate_sample_name(row, id)
    sample_names.append(sample_name)

metadata['sample_name'] = sample_names

### CREATE HERE a new dataframe with the following columns: sample, fastq_1, fastq_2.  sample will be equal to sample_name, fastq_1 will be equal to {Run}_1.fastq.gz and fastq_2 will be equal to {Run}_2.fastq.gz

#metadata.to_csv('metadata_with_sample_names.csv', index=False)


/tmp/ipykernel_127128/172926178.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metadata['sample_name'] = sample_names


,Run,treatment,tra-1-60,sample_name
0,SRR5739552,WT,NaN,WT_1
1,SRR5739553,WT,NaN,WT_2
2,SRR5739554,WT,NaN,WT_3
3,SRR5739555,WT,NaN,WT_4
4,SRR5739556,WT,NaN,WT_5
...,...,...,...,...
475,SRR5740027,BJ fibroblasts 16 days post induced viral expr...,Negative,BJ_fibroblasts_16_92_Negative
476,SRR5740028,BJ fibroblasts 16 days post induced viral expr...,Negative,BJ_fibroblasts_16_93_Negative
477,SRR5740029,BJ fibroblasts 16 days post induced viral expr...,Negative,BJ_fibroblasts_16_94_Negative
478,SRR5740030,BJ fibroblasts 16 days post induced viral expr...,Negative,BJ_fibroblasts_16_95_Negative


# This is the one that works for GSE100344

In [14]:
import pandas as pd

# Load the SraRunTable.txt file with the correct delimiter
sra_run_table = pd.read_csv('SraRunTable.txt', sep=',')

# Extract relevant columns
metadata = sra_run_table[['Run', 'treatment', 'tra-1-60']]

# Generate the sample_name column
def generate_sample_name(row, id):
    treatment_words = row['treatment'].split()[:3]
    sample_name = '_'.join(treatment_words) + f'_{id}'
    if pd.notna(row['tra-1-60']) and row['tra-1-60'].strip() != '':
        sample_name += f"_{row['tra-1-60']}"
    return sample_name

# Add the sample_name column
sample_names = []
for idx, row in metadata.iterrows():
    id = (idx % 96) + 1
    sample_name = generate_sample_name(row, id)
    sample_names.append(sample_name)

metadata['sample_name'] = sample_names

# Create the new dataframe with columns: sample, fastq_1, fastq_2
new_df = pd.DataFrame({
    'sample': metadata['sample_name'],
    'fastq_1': metadata['Run'] + '_1.fastq.gz',
    'fastq_2': metadata['Run'] + '_2.fastq.gz'
})

new_df.to_csv('samplesheet.csv', index=False)

FileNotFoundError: [Errno 2] No such file or directory: 'SraRunTable.txt'

In [8]:
import os
import pandas as pd

file_path = './samplesheet.csv'
samplesheet = pd.read_csv(file_path)

# Check for missing files
missing_files = {'fastq_1': [], 'fastq_2': []}

for index, row in samplesheet.iterrows():
    if not os.path.isfile(row['fastq_1']):
        missing_files['fastq_1'].append(row['fastq_1'])
    if not os.path.isfile(row['fastq_2']):
        missing_files['fastq_2'].append(row['fastq_2'])

missing_files

srr_ids = [filename.split('_')[0] for filename in missing_files['fastq_1']]

print(srr_ids)
for srr_id in srr_ids:
    print(srr_id)



['SRR5739612', 'SRR5739613', 'SRR5739614', 'SRR5739615', 'SRR5739616', 'SRR5739618', 'SRR5739619', 'SRR5739620', 'SRR5739621', 'SRR5739622', 'SRR5739623', 'SRR5739716', 'SRR5739717', 'SRR5739718', 'SRR5739719', 'SRR5739720', 'SRR5739721', 'SRR5739722', 'SRR5739723', 'SRR5739724', 'SRR5739725', 'SRR5739726', 'SRR5739727', 'SRR5739728', 'SRR5739729', 'SRR5739730', 'SRR5739731']
SRR5739612
SRR5739613
SRR5739614
SRR5739615
SRR5739616
SRR5739618
SRR5739619
SRR5739620
SRR5739621
SRR5739622
SRR5739623
SRR5739716
SRR5739717
SRR5739718
SRR5739719
SRR5739720
SRR5739721
SRR5739722
SRR5739723
SRR5739724
SRR5739725
SRR5739726
SRR5739727
SRR5739728
SRR5739729
SRR5739730
SRR5739731


In [3]:
# Remove rows with missing files from the samplesheet
missing_fastq_1 = set(missing_files['fastq_1'])
missing_fastq_2 = set(missing_files['fastq_2'])

# Filter out the rows with missing files
filtered_samplesheet = samplesheet[~samplesheet['fastq_1'].isin(missing_fastq_1) & ~samplesheet['fastq_2'].isin(missing_fastq_2)]

# Save the filtered samplesheet to a new CSV file
filtered_file_path = './filtered_samplesheet.tsv'
filtered_samplesheet.to_csv(filtered_file_path, index=False, sep='\t')

filtered_samplesheet


,sample,fastq_1,fastq_2
0,WT_1,SRR5739552_1.fastq.gz,SRR5739552_2.fastq.gz
1,WT_2,SRR5739553_1.fastq.gz,SRR5739553_2.fastq.gz
2,WT_3,SRR5739554_1.fastq.gz,SRR5739554_2.fastq.gz
3,WT_4,SRR5739555_1.fastq.gz,SRR5739555_2.fastq.gz
4,WT_5,SRR5739556_1.fastq.gz,SRR5739556_2.fastq.gz
...,...,...,...
475,BJ_fibroblasts_16_92_Negative,SRR5740027_1.fastq.gz,SRR5740027_2.fastq.gz
476,BJ_fibroblasts_16_93_Negative,SRR5740028_1.fastq.gz,SRR5740028_2.fastq.gz
477,BJ_fibroblasts_16_94_Negative,SRR5740029_1.fastq.gz,SRR5740029_2.fastq.gz
478,BJ_fibroblasts_16_95_Negative,SRR5740030_1.fastq.gz,SRR5740030_2.fastq.gz


In [6]:
# Reorder columns to move the 'sample' column to the end
cols = filtered_samplesheet.columns.tolist()
cols.append(cols.pop(cols.index('sample')))
filtered_samplesheet = filtered_samplesheet[cols]

# Save the reordered samplesheet to a new TSV file
reordered_tsv_file_path = './reordered_samplesheet.tsv'
filtered_samplesheet.to_csv(reordered_tsv_file_path, index=False, sep='\t')

filtered_samplesheet


,fastq_1,fastq_2,sample
0,SRR5739552_1.fastq.gz,SRR5739552_2.fastq.gz,WT_1
1,SRR5739553_1.fastq.gz,SRR5739553_2.fastq.gz,WT_2
2,SRR5739554_1.fastq.gz,SRR5739554_2.fastq.gz,WT_3
3,SRR5739555_1.fastq.gz,SRR5739555_2.fastq.gz,WT_4
4,SRR5739556_1.fastq.gz,SRR5739556_2.fastq.gz,WT_5
...,...,...,...
475,SRR5740027_1.fastq.gz,SRR5740027_2.fastq.gz,BJ_fibroblasts_16_92_Negative
476,SRR5740028_1.fastq.gz,SRR5740028_2.fastq.gz,BJ_fibroblasts_16_93_Negative
477,SRR5740029_1.fastq.gz,SRR5740029_2.fastq.gz,BJ_fibroblasts_16_94_Negative
478,SRR5740030_1.fastq.gz,SRR5740030_2.fastq.gz,BJ_fibroblasts_16_95_Negative


In [8]:
import os

# Load the reordered samplesheet TSV file
reordered_tsv_file_path = './reordered_samplesheet.tsv'
samplesheet = pd.read_csv(reordered_tsv_file_path, sep='\t')

# Check for missing files and permissions
missing_files = {'fastq_1': [], 'fastq_2': []}
permission_issues = {'fastq_1': [], 'fastq_2': []}

for index, row in samplesheet.iterrows():
    if not os.path.isfile(row['fastq_1']):
        missing_files['fastq_1'].append(row['fastq_1'])
    elif not os.access(row['fastq_1'], os.R_OK):
        permission_issues['fastq_1'].append(row['fastq_1'])
    
    if not os.path.isfile(row['fastq_2']):
        missing_files['fastq_2'].append(row['fastq_2'])
    elif not os.access(row['fastq_2'], os.R_OK):
        permission_issues['fastq_2'].append(row['fastq_2'])

missing_files, permission_issues

({'fastq_1': [], 'fastq_2': []}, {'fastq_1': [], 'fastq_2': []})

# Rename from SRR{ID}_{1|2}.fastq.gz to SAMPLE 

In [9]:
import os
import pandas as pd

# Load the samplesheet CSV file
file_path = 'samplesheet.csv'
samplesheet = pd.read_csv(file_path, header=None, names=['sample', 'fastq_1', 'fastq_2'])

# Loop through each row in the samplesheet and rename the files
for index, row in samplesheet.iterrows():
    sample_name = row['sample']
    old_fastq_1 = row['fastq_1']
    old_fastq_2 = row['fastq_2']
    new_fastq_1 = f"{sample_name}_1.fastq.gz"
    new_fastq_2 = f"{sample_name}_2.fastq.gz"
    
    # Rename the files
    if os.path.isfile(old_fastq_1):
        os.rename(old_fastq_1, new_fastq_1)
    if os.path.isfile(old_fastq_2):
        os.rename(old_fastq_2, new_fastq_2)

    print(f"Renamed {old_fastq_1} to {new_fastq_1}")
    print(f"Renamed {old_fastq_2} to {new_fastq_2}")

# Save the updated samplesheet with new names
samplesheet['fastq_1'] = samplesheet['sample'] + '_1.fastq.gz'
samplesheet['fastq_2'] = samplesheet['sample'] + '_2.fastq.gz'
samplesheet.to_csv('updated_samplesheet.csv', index=False, header=False)


Renamed fastq_1 to sample_1.fastq.gz
Renamed fastq_2 to sample_2.fastq.gz
Renamed SRR5739552_1.fastq.gz to WT_1_1.fastq.gz
Renamed SRR5739552_2.fastq.gz to WT_1_2.fastq.gz
Renamed SRR5739553_1.fastq.gz to WT_2_1.fastq.gz
Renamed SRR5739553_2.fastq.gz to WT_2_2.fastq.gz
Renamed SRR5739554_1.fastq.gz to WT_3_1.fastq.gz
Renamed SRR5739554_2.fastq.gz to WT_3_2.fastq.gz
Renamed SRR5739555_1.fastq.gz to WT_4_1.fastq.gz
Renamed SRR5739555_2.fastq.gz to WT_4_2.fastq.gz
Renamed SRR5739556_1.fastq.gz to WT_5_1.fastq.gz
Renamed SRR5739556_2.fastq.gz to WT_5_2.fastq.gz
Renamed SRR5739557_1.fastq.gz to WT_6_1.fastq.gz
Renamed SRR5739557_2.fastq.gz to WT_6_2.fastq.gz
Renamed SRR5739558_1.fastq.gz to WT_7_1.fastq.gz
Renamed SRR5739558_2.fastq.gz to WT_7_2.fastq.gz
Renamed SRR5739559_1.fastq.gz to WT_8_1.fastq.gz
Renamed SRR5739559_2.fastq.gz to WT_8_2.fastq.gz
Renamed SRR5739560_1.fastq.gz to WT_9_1.fastq.gz
Renamed SRR5739560_2.fastq.gz to WT_9_2.fastq.gz
Renamed SRR5739561_1.fastq.gz to WT_10_1.fas

# CREATE SAMPLESHEET.TSV GIVEN FILENAMES

This code will create a samplesheet.tsv following the next steps:
1. It will check all the files *fastq.gz in a given folder.
2. It will divide the name in {NAME}_{1|2}.fastq.gz
3. It will create a samplesheet.tsv with three columns with the following information: {NAME}_1.fastq.gz {NAME}_2.fastq.gz {NAME}

Due to this approach, it will only contain information regarding the files inside of a given folder, avoiding any issue due to non existing files.

**Notes:** Please notice that new file called `samplesheet.tsv` will be saved into the directory where we are analysing the fastq files, not where jupyter notebook file is located.

In [13]:
import os
import pandas as pd

def generate_samplesheet(directory):
    # Initialize an empty list to store sample data
    samples = []

    # Loop through files in the directory
    for file in os.listdir(directory):
        if file.endswith('_1.fastq.gz'):
            # Extract the base name (without _1.fastq.gz)
            base_name = file.rsplit('_', 1)[0]
            fastq_1 = file
            fastq_2 = base_name + '_2.fastq.gz'
            
            # Check if the corresponding _2.fastq.gz file exists
            if os.path.isfile(os.path.join(directory, fastq_2)):
                samples.append([base_name, fastq_1, fastq_2])

    # Create a DataFrame and save as TSV without header
    samplesheet = pd.DataFrame(samples, columns=['Fastq_1', 'Fastq_2', 'Name'])
    samplesheet.to_csv(os.path.join(directory, 'samplesheet.tsv'), sep='\t', index=False, header=False)

# Specify the directory containing the fastq.gz files
directory = './D8/'

# Generate the samplesheet
generate_samplesheet(directory)